# Segmentation

## Importing dependencies

In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import json
import openslide as op
from PIL import Image
from util import IoU, Dice, masked, remove_small_regions, normalize
from keras.models import load_model, model_from_json

from sklearn.utils.validation import check_array
import numbers
from itertools import product
from scipy import sparse
from numpy.lib.stride_tricks import as_strided

from scipy import misc
import gc

Using TensorFlow backend.


## Configuration

In [2]:
# Paths
filename = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.svs'
maskname = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.png'

# Do not touch
level=1
position=(0,0)
im_shape = (256, 256,3) # Shape of the image (patch)
pa_shape = (256, 256)   # Shape of the mask (patch)

## Image loading

### Loading base picture

In [3]:
im = op.OpenSlide(filename)
imload = np.asfarray(im.read_region((0,0), 1, im.level_dimensions[1]),dtype=np.float32)[:,:,0:3]/255

In [4]:
print(imload.mean())
print(imload.std())

0.806298
0.176316


### Loading mask

In [5]:
maskload = Image.open(maskname)
maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[level], Image.ANTIALIAS),dtype=np.float32),-1)

### Allocating final array

In [6]:
#output = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.float32)

In [7]:
outputThreshold = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0], 3),dtype=np.int8)

## Loading model

In [8]:
# Load model
model_name = './model.099.hdf5'
UNet = load_model(model_name)

## Execution

### Initialization  

In [ ]:
test_gen = ImageDataGenerator(rescale=1.)
#ious = np.zeros(n_test)
#dices = np.zeros(n_test)
#gts, prs = [], []
i = 0
iterX=0
iterY=0

im_mean=imload.mean()
im_std=imload.std()
#plt.figure(figsize=(10, 10))

inp_shape=(256,256,3)

### Prediction

In [ ]:

# Prediction
#for xx, yy in test_gen.flow(X, y, batch_size=1):
for h in range(0,im.level_dimensions[level][0]-256,256):
    for j in range(0,im.level_dimensions[level][1]-256,256):
        posX=h
        posY=j
        
        xx = np.expand_dims(imload[posY:posY+256,posX:posX+256,:],0)
        xx -= im_mean
        xx /= im_std
        yy = maskload[posY:posY+256,posX:posX+256,:]
        #print(xx.shape)
        #print(yy.shape)
        #xx, yy = np.expand_dims(X[i],0),np.expand_dims(y[i],0)
        #print(xx.shape)
        #print(yy.shape)
        
        #img = exposure.rescale_intensity(np.squeeze(xx), out_range=(0,1))
        pred = UNet.predict(xx)[..., 0].reshape(inp_shape[:2])
        mask = yy[..., 0].reshape(inp_shape[:2])
        gt = mask > 0.5
        pr = pred > 0.5
        pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
        #io.imsave('{}'.format(df.iloc[i].path), masked(img, gt, pr, 1))
        #gts.append(gt)
        #prs.append(pr)
        #ious[i] = IoU(gt, pr)
        #dices[i] = Dice(gt, pr)
        #print(i, ious[i], dices[i])
        
        #outputPatches[i,:,:,:] = np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8)))
        outputPart = np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8)))
        #if i < 4:
        #    plt.subplot(4, 4, 4*i+1)
        #    plt.title('Processed ' + str(i))
        #    plt.axis('off')
        #    plt.imshow(img, cmap='gray')
        #    plt.subplot(4, 4, 4 * i + 2)
        #    plt.title('IoU = {:.4f}'.format(ious[i]))
        #    plt.axis('off')
        #    plt.imshow(masked(img, gt, pr, 1))
        #    plt.subplot(4, 4, 4*i+3)
        #    plt.title('Prediction')
        #    plt.axis('off')
        #    plt.imshow(pred, cmap='jet')
        #    plt.subplot(4, 4, 4*i+4)
        #    plt.title('Difference')
        #    plt.axis('off')
        #    plt.imshow(outputPart)

        #Add the prediction patch to the output picture


        #if i!=0 and i % modulo == 0:
        #    iterX=0
        #    iterY+=1


        part=outputThreshold[posY:posY+256,posX:posX+256,:]
        part[:,:,:]=outputPart[0:part.shape[0],0:part.shape[1],:]

        #part=output[posY:posY+256,posX:posX+256]
        #part[:,:]=pred[0:part.shape[0],0:part.shape[1]]

        #iterX+=1

        i += 1
        #if i == n_test:
        #    break

 
#print('Mean IoU:', ious.mean())
#print('Mean Dice:', dices.mean())
#plt.tight_layout()
#plt.savefig('results.png')
#plt.show()

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(imload,interpolation='none')
plt.show()

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(outputThreshold,interpolation='none', cmap="jet")
plt.show()

In [23]:
misc.imsave("prediction_15-02_voley.png", outputThreshold)